In [1]:
import numpy as np
import os
# import cv2
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, adam
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.densenet import DenseNet121

# 랜덤시드 고정시키기
np.random.seed(3)

Using TensorFlow backend.


In [2]:
# import cv2

In [4]:
model_2 = DenseNet121(include_top=True, weights = None, input_tensor=None, input_shape=(224,224,3), pooling=None, classes=7)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32 # 16->64->32
epochs = 100 # 50->100
train_path = '/home/mywork/kijun_kwon/data_split1/train'
test_path = '/home/mywork/kijun_kwon/data_split1/test'
val_path = '/home/mywork/kijun_kwon/data_split1/val'
target_size = 224
num_train = 1965
num_test = 658
num_val = 655

class_root='/home/mywork/kijun_kwon/data_split1/train'
class_list = [ item for item in os.listdir(class_root) if os.path.isdir(os.path.join(class_root, item)) ]

In [6]:
train_generator = train_datagen.flow_from_directory(
        train_path,
        color_mode = "rgb",
        target_size=(target_size, target_size), #높을수록 이미지 손실 줄임.
        batch_size = batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_path,
        color_mode = "rgb",
        target_size=(target_size, target_size),    
        batch_size=batch_size,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        val_path,
        color_mode = "rgb",
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='categorical')

Found 1965 images belonging to 7 classes.
Found 658 images belonging to 7 classes.
Found 655 images belonging to 7 classes.


In [7]:
target_size

224

In [9]:
model = model_2

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model_checkpoint = ModelCheckpoint(('./densenet.{epoch:02d}.hdf5'),
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=True)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                         patience=2, verbose=1)

callbacks = [model_checkpoint, reduce_learning_rate]

In [12]:
model.fit_generator(
        train_generator,
        steps_per_epoch = num_train // batch_size,
        epochs=10,
        validation_data=val_generator,
        validation_steps= num_val // batch_size)

Epoch 1/10
61/61 [==============================] - 44s 727ms/step - loss: 0.9792 - acc: 0.6534 - val_loss: 10.7104 - val_acc: 0.2844
Epoch 2/10
61/61 [==============================] - 18s 296ms/step - loss: 0.6638 - acc: 0.7715 - val_loss: 14.5244 - val_acc: 0.0979
Epoch 3/10
61/61 [==============================] - 18s 296ms/step - loss: 0.6708 - acc: 0.7660 - val_loss: 10.6683 - val_acc: 0.1669
Epoch 4/10
61/61 [==============================] - 18s 287ms/step - loss: 0.5851 - acc: 0.7845 - val_loss: 10.3197 - val_acc: 0.1685
Epoch 5/10
61/61 [==============================] - 18s 290ms/step - loss: 0.4320 - acc: 0.8374 - val_loss: 13.4275 - val_acc: 0.1027
Epoch 6/10
61/61 [==============================] - 18s 290ms/step - loss: 0.3900 - acc: 0.8555 - val_loss: 10.3251 - val_acc: 0.2809
Epoch 7/10
61/61 [==============================] - 18s 288ms/step - loss: 0.4493 - acc: 0.8399 - val_loss: 1.7894 - val_acc: 0.5313
Epoch 8/10
61/61 [==============================] - 17s 286ms/s

ImportError: No module named 'cv2'

In [18]:
# out = model.predict(im)
# print (np.argmax(out))

In [13]:
Y_pred = model.predict_generator(test_generator, num_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=class_list))

Confusion Matrix
[[ 39   1   0  20   0  66   2]
 [ 28   0   0  10   0  36   0]
 [ 73   1   0  23   0 128   0]
 [ 32   0   0   6   0  31   0]
 [  8   0   0   1   0   9   0]
 [ 30   0   0   9   0  50   0]
 [ 18   0   0   9   0  28   0]]
Classification Report
               precision    recall  f1-score   support

         sink       0.17      0.30      0.22       128
           ok       0.00      0.00      0.00        74
         dent       0.00      0.00      0.00       225
      scratch       0.08      0.09      0.08        69
    substance       0.00      0.00      0.00        18
        other       0.14      0.56      0.23        89
discoloration       0.00      0.00      0.00        55

    micro avg       0.14      0.14      0.14       658
    macro avg       0.06      0.14      0.08       658
 weighted avg       0.06      0.14      0.08       658



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 35.00%
